In [1]:
# Import necessary libraries
import random
from tensorflow.keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model, Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer

In [3]:
# Initialize WordNetLemmatizer and download NLTK data
lemmatizer = WordNetLemmatizer()
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Abdulmunim\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Abdulmunim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Abdulmunim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# Initialize variables and read data from intents.json
words = []
classes = []
documents = []
ignore_words = ["?", "!"]
data_file = open("intents.json").read()
intents = json.loads(data_file)

In [7]:
# Process intents and extract words and classes
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        # Tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Add documents
        documents.append((w, intent["tag"]))

        # Add classes to the class list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [8]:
# Lemmatize words and create a sorted unique word list
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))


In [9]:
# Sort classes
classes = sorted(list(set(classes)))

In [10]:
# Print some statistics
print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

100 documents
55 classes ['AI', 'abbr', 'artificial', 'bend', 'body', 'bot1', 'breathe', 'business', 'chatbot', 'chatterbox', 'clone', 'comp', 'computer', 'control', 'cramped', 'date', 'death', 'do', 'events', 'fav', 'fight', 'goodbye', 'greetings', 'hardware', 'hobby', 'idea', 'imortal', 'lang', 'laugh', 'lie', 'machine', 'malfunction', 'motormouth', 'move', 'name', 'name1', 'need', 'noanswer', 'os', 'program', 'programming', 'ratchet', 'robotics', 'robots', 'robotss', 'sapient', 'sense', 'sentiment', 'shoe', 'sound', 'stupid', 'thanks', 'usage', 'who', 'wt']
126 unique lemmatized words ["'m", "'s", ',', 'a', 'ai', 'all', 'allowed', 'am', 'an', 'are', 'artificial', 'awesome', 'be', 'being', 'bend', 'body', 'bot', 'breathe', 'business', 'bye', 'can', 'chat', 'chatterbox', 'clone', 'coffee', 'computer', 'control', 'cramped', 'data', 'date', 'die', 'do', 'entity', 'event', 'favorite', 'favour', 'fight', 'for', 'good', 'great', 'hardware', 'haroo', 'hello', 'help', 'helpful', 'helping', '

In [11]:
# Save words and classes to pickle files
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [12]:
# Initialize training data
training = []
output_empty = [0] * len(classes)

In [13]:
# Process documents to create a bag of words and output labels
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [14]:
# Shuffle training data
random.shuffle(training)

In [15]:
# Separate bag-of-words representations and output labels
train_x = [item[0] for item in training]
train_y = [item[1] for item in training]

In [16]:
# Convert to NumPy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)

print("Training data created")


Training data created


In [17]:
# Create a neural network model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

# Compile the model using SGD optimizer
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               16256     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 55)                3575      
                                                                 
Total params: 28087 (109.71 KB)
Trainable params: 28087 (109.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# Fit the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
20/20 [==============================] - 1s 3ms/step - loss: 4.0134 - accuracy: 0.0300
Epoch 2/200
20/20 [==============================] - 0s 3ms/step - loss: 3.9495 - accuracy: 0.0700
Epoch 3/200
20/20 [==============================] - 0s 2ms/step - loss: 3.8714 - accuracy: 0.1100
Epoch 4/200
20/20 [==============================] - 0s 3ms/step - loss: 3.8280 - accuracy: 0.1000
Epoch 5/200
20/20 [==============================] - 0s 2ms/step - loss: 3.7431 - accuracy: 0.0900
Epoch 6/200
20/20 [==============================] - 0s 2ms/step - loss: 3.6472 - accuracy: 0.1200
Epoch 7/200
20/20 [==============================] - 0s 2ms/step - loss: 3.5066 - accuracy: 0.1700
Epoch 8/200
20/20 [==============================] - 0s 2ms/step - loss: 3.4809 - accuracy: 0.1500
Epoch 9/200
20/20 [==============================] - 0s 2ms/step - loss: 3.4152 - accuracy: 0.1300
Epoch 10/200
20/20 [==============================] - 0s 2ms/step - loss: 3.2620 - accuracy: 0.1800
Epoch 11/

In [19]:
# Save the model
model.save("chatbot_model.h5", hist)
print("Model created")

Model created


c:\Users\Abdulmunim\Desktop\SYNC\AAU-Chatbot\engine\.venv\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
